# Quantum Noise-Based Map Generation

### By: Michael Wornow and Vishal Jain _(ES 170 Final Project)_

![title](Images/terrain.png)

## 1. Motivation (Michael)

Many video games rely on procedural map generation algorithms in order to create exciting and novel worlds for the player to explore. The advantages of such algorithms are several-fold: 

1. **More addictive** - They increase the re-play value of a game by ensuring that every restart results in a slightly different map.
5. **More natural** - Procedural map algorithms strike the perfect balance between purely random maps and highly artificial/man-made-looking maps, resulting in a natural-looking and interesting terrain.
2. **Save time** - They are entirely programmatic, allowing developers to spend time improving other aspects of the game instead of laboring over the design of each pixel of a terrain.
3. **Quicker changes** - They allow for fine-tuning of map characteristics (i.e. elevation, moisture, ocean level, etc.) that take immediate effect across the entire world, without having to modify every single pixel that would be affected by the change. 
4. **Save memory** - Because these algorithms are deterministic given a specific seed, they can save memory by only generating map features as a player explores the relevant sections of a world and thereby avoid having to save the entire world in memory.

Almost every video game today that relies on generating semi-random maps or texturing natural terrain uses some form of procedural map generation to achieve these goals. The most popular example of such a game is Microsoft's Minecraft, which uses several layers of procedural algorithms in order to generate vast, unpredictable worlds that still have enough structure and consistency to make them entertaining to play. [https://www.engadget.com/2015/03/04/how-minecraft-worlds-are-made/]

![title](Images/minecraft.jpg)

The process of generating a map is generally as follows: start by generating many layers of some sort of noise at different frequencies, then layer them together with different weights, and finally map the resulting matrix to "elevations" or "moisture" or "biomes" or simply pixel colors on your map. Thus, the key behind procedural map generating algorithms are the noising functions that they use.

![title](Images/perlin_progression.png)

The most famous such noise generation algorithm is called "Perlin noise," and was created in 1983 by Ken Perlin in order to make computer graphics of natural phenomenon appear less "machine-like." [Perlin, Ken (July 1985). "An Image Synthesizer". SIGGRAPH Comput. Graph. 19 (0097–8930): 287–296. doi:10.1145/325165.325247. Retrieved 9 February 2016.] The algorithm is explained in detail later in this project, as well as modifications that have been subsequently proposed to improve its noising properties and computational tractability.

While there are currently many other "classical" noise generation algorithms being applied to computer graphics and simulation (including Simplex noise, Worley noise, simulation noise, value noise, etc.), we hadn't encountered any applied usage of "quantum" noising towards computer graphics or procedural map generation. Given the fact that noise is a central part of quantum computing in the real world, and quantum phenomenon oftentimes defy human expectations of reality even in the absence of such measurement noise, it made sense that a procedural map generating algorithm based on quantum noise could offer a novel approach towards generating interesting, natural-looking, and unpredictable maps.

Thus, the goal of this project was to take advantage of quantum phenomenon like entanglement and superposition, as well as the actual measurement noise associated with making measurements on real-world quantum computers, in order to procedurally generate interesting maps.

The project is split into five main segments:

1. **Motivation**: Project goal and motivation
1. **Perlin Noise**: An explanation and implementation of the classical Perlin noise algorithm, an Exponential version of Perlin noise, and a first attempt at creating a version runnable on a quantum computer
1. **Map Generation**: Code to turn a matrix of noise into a procedurally generated map
1. **Quantum Circuits**: Code containing QiSkit quantum circuits for generating noise
1. **Evaluation**: Quantifying the noise generated by the different QiSkit circuits tested
1. **Conclusion**: Short conclusion summarizing our results, lessons learned, and future directions.

In [1]:
# Requirements:
# numpy, matplotlib, qiskit, pillow, scipy, scikit-image, noise, qutip

In [2]:
# Imports
import numpy as np
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
from qiskit import *
from time import time
from PIL import Image
from scipy.signal import convolve2d
from skimage.restoration import estimate_sigma
from scipy.misc import toimage
import noise as python_noise
from qutip import *

# Ignore scipy toimage deprecation warning
import warnings
warnings.filterwarnings("ignore")

In [3]:
def display_image(arr):
    # Visualize noise as 2D grayscale plot
    toimage(arr).show()

## 2. Perlin Noise (Vishal)

A good random number generator produces numbers that have no relationship and show no discernible pattern. Some randomness is good if the randomness is organic. However, randomness as a guiding principle is not natural. The Perlin Noise algorithm is developed to create more natural noise.

Perlin noise then produces pseudo-random noise - naturally ordered smooth sequence of numbers. The steps to the algorithms are as follows:

1. In an n-dimensional grid, each point has a random n-dimensional unit-length gradient vector. In one dimension, the gradients are just random scalars between -1 and 1.

2. Given an n-dimensional gradient value at each grid node, the next step in the algorithm is to determine into which grid cell the given point falls. First, we determine the distance vector from the corner node and the point. Then, dot product between gradient vector at the node and the distance vector. For a point in a two-dimensional grid, this will require the computation of 4 distance vectors and dot products, while in three dimensions 8 distance vectors and 8 dot products are needed. This leads to the $O(2^n)$ complexity scaling.

![title](Images/perlin_noise.png)

3. Use some sort of function to smooth out with interpolation. Gives Perlin algorithms its spatial appeal.

We used Perlin Noise algorithm as a basis of comparison with our quantum noise generation techniques. We implemented a normal Perlin Noise algorithm, one in Qiskit, and an exponential Perlin Algorithm (variant) which has more appealing gradients. Because the Perlin Algorithm is exponential, it would be a reasonable next step to implement a quantum version of the algorithm.

In [4]:
def generate_gradient(seed=None):
    global gradient
    seed and np.random.seed(seed)
    gradient = np.random.rand(512, 512, 2) * 2 - 1

def perlin_noise(size_x, size_y, frequency):
    global gradient
    # linear space by frequency
    x = np.tile(
        np.linspace(0, frequency, size_x, endpoint=False), 
        size_y
    )
    y = np.repeat(
        np.linspace(0, frequency, size_y, endpoint=False), 
        size_x
    )
    # gradient coordinates
    x0 = x.astype(int)
    y0 = y.astype(int)
    # local coordinate
    x -= x0
    y -= y0
    # gradient projections
    g00 = gradient[x0, y0]
    g10 = gradient[x0 + 1, y0]
    g01 = gradient[x0, y0 + 1]
    g11 = gradient[x0 + 1, y0 + 1]
    # fade
    t = (3 - 2 * x) * x * x
    xp = Qobj(t)
    test = xp*xp.trans()
    # linear interpolation
    r = g00[:, 0] * x + g00[:, 1] * y
    s = g10[:, 0] * (x - 1) + g10[:, 1] * y
    g0 = r + t * (s - r)
    # linear interpolation
    r = g01[:, 0] * x + g01[:, 1] * (y - 1)
    s = g11[:, 0] * (x - 1) + g11[:, 1] * (y - 1)
    g1 = r + t * (s - r)
    # fade
    t = (3 - 2 * y) * y * y
    # (bi)linear interpolation
    g = g0 + t * (g1 - g0)
    # reshape
    return g.reshape(size_y, size_x)

def banded_perlin_noise(size_x, size_y, frequencies, amplitudes):
    image = np.zeros((size_y, size_x))
    for f, a in zip(frequencies, amplitudes):
        image += perlin_noise(size_x, size_y, f) * a
    image -= image.min()
    image /= image.max()
    return image
  
def exponential_perlin_noise(size_x, size_y, frequency):
    global gradient
    # linear space by frequency
    x = np.tile(
        np.linspace(0, frequency, size_x, endpoint=False), 
        size_y
    )
    y = np.repeat(
        np.linspace(0, frequency, size_y, endpoint=False), 
        size_x
    )
    # gradient coordinates
    x0 = x.astype(int)
    y0 = y.astype(int)
    # local coordinate
    x -= x0
    y -= y0
    # gradient projections
    g00 = gradient[x0, y0]
    g10 = gradient[x0 + 1, y0]
    g01 = gradient[x0, y0 + 1]
    g11 = gradient[x0 + 1, y0 + 1]
    # fade
    t = (3 - 2 * x) * x * x
    xp = Qobj(t)
    test = xp*xp.trans()
    # linear interpolation
    test = np.absolute(np.random.exponential(scale=2.0, size=(size_x*size_y,)))
    np.clip(test, 0, 1, out=test)
    r = test*g00[:, 0] * x + test*g00[:, 1] * y
    s = test*g10[:, 0] * (x - 1) + test*g10[:, 1] * y
    g0 = r + t * (s - r)
    # linear interpolation
    r = test*g01[:, 0] * x + test*g01[:, 1] * (y - 1)
    s = test*g11[:, 0] * (x - 1) + test*g11[:, 1] * (y - 1)
    g1 = r + t * (s - r)
    # fade
    t = (3 - 2 * y) * y * y
    # (bi)linear interpolation
    g = g0 + t * (g1 - g0)
    # reshape
    return g.reshape(size_y, size_x)

def quantum_perlin_noise(size_x, size_y, frequency):
    global gradient
    # linear space by frequency
    x = np.tile(
        np.linspace(0, frequency, size_x, endpoint=False), 
        size_y
    )
    y = np.repeat(
        np.linspace(0, frequency, size_y, endpoint=False), 
        size_x
    )
    # gradient coordinates
    x0 = x.astype(int)
    y0 = y.astype(int)
    # local coordinate
    x -= x0
    y -= y0
    # gradient projections
    g00 = gradient[x0, y0]
    g10 = gradient[x0 + 1, y0]
    g01 = gradient[x0, y0 + 1]
    g11 = gradient[x0 + 1, y0 + 1]
    # fade quantum
    x_alt = Qobj(x)
    y_alt = Qobj(y)
    t_ans = (Qobj(np.repeat(3,25)) - 2*x_alt).trans()*x_alt*x_alt.trans()
    #linear interpolation quantum: 
    r_alt = Qobj(g00[:,0]).trans()*x_alt + Qobj(g00[:, 1]).trans()*y_alt
    s_alt = Qobj(g10[:,0]).trans()*(x_alt - 1) + Qobj(g10[:, 1]).trans()*y_alt
    g0_alt = r_alt + t_ans*(s_alt - r_alt)
    # linear interpolation quantum: 
    r_alt = Qobj(g01[:,0]).trans()*x_alt + Qobj(g01[:, 1]).trans()*(y_alt - 1)
    s_alt = Qobj(g11[:,0]).trans()*(x_alt - 1) + Qobj(g11[:, 1]).trans()*(y_alt-1)
    g1_alt = r_alt + t_ans*(s_alt - r_alt)
    # fade quantum
    t_alt = (3 - 2*y_alt).trans()*y_alt*y_alt.trans()
    g_alt = (g0_alt + t_alt*(g1_alt - g0_alt).trans()).full()
    # reshape
    return g_alt.reshape(size_y, size_x)

In [5]:
generate_gradient()

image = perlin_noise(124, 124, 2)
display_image(image)

## 3. Map Generation (Michael)

The code in this section procedurally generates a map based on a matrix of noise.

First, the various possible biomes that our map can contain (i.e. "Ocean", "Beach", "Grassland", etc.) are defined and assigned a unique RGB color code.

In [6]:
# Scale for classical noise maps
CLASSICAL_SCALE = (1024, 1024)

# Biome segmentation and coloring
OCEAN = [68, 68, 122]
COAST = [51, 51, 90]
LAKESHORE = [34, 85, 136]
LAKE = [51, 102, 153]
RIVER = [34, 85, 136]
MARSH = [47, 102, 102]
ICE = [153, 255, 255]
BEACH = [160, 144, 119]
BRIDGE = [104, 104, 96]
LAVA = [204, 51, 51]
SNOW = [221, 221, 228]
TUNDRA = [187, 187, 170]
BARE = [136, 136, 136]
SCORCHED = [85, 85, 85]
TAIGA = [153, 170, 119]
SHRUBLAND = [136, 153, 119]
TEMPERATE_DESERT = [201, 210, 155]
TEMPERATE_RAIN_FOREST = [68, 136, 85]
TEMPERATE_DECIDUOUS_FOREST = [103, 148, 89]
GRASSLAND = [136, 170, 85]
SUBTROPICAL_DESERT = [210, 185, 139]
TROPICAL_RAIN_FOREST = [51, 119, 85]
TROPICAL_SEASONAL_FOREST = [85, 153, 68]

Now that the biome colors are defined I needed to write a function that mapped noise to each biome.

The two functions below each take two parameter, $e$ ("elevation") and $m$ ("moisture"), and return a specific biome. Each parameter has a range of $[0,1]$ (this value is determined by the noise function) and corresponds to a specific $(x,y)$ coordinate in our world.

The **biome_basic** function ignores the moisture $m$ parameter, and thus defines biomes purely based on their elevation $e$. The **biome_advanced** function, on the other hand, takes into account both elevation $e$ and moisture $m$ noise in order to create more random/fine-grained biomes.

For example, if the noise value at coordinate $(10,10)$ was $0.09$, and this value of $0.09$ was passed to **biome_basic(0.09, None)**, then the coordinate $(10,10)$ in our world would be colored **OCEAN**. This makes intuitive sense because the lower elevations of our map should be below sea level. However, if the noise value at the point $(10,10)$ was $0.9$ then it would be colored **SNOW**, which also makes sense since higher elevations have snow.

In [7]:
def biome_basic(e, m):
    if (e < 0.1):
        return OCEAN
    elif (e < 0.12):
        return BEACH
    elif (e < 0.25):
        return TEMPERATE_DECIDUOUS_FOREST
    elif (e < 0.4):
        return TEMPERATE_RAIN_FOREST
    elif (e < 0.65):
        return TAIGA
    elif (e < 0.85):
        return TUNDRA
    else:
        return SNOW

def biome_advanced(e, m):
    ocean_baseline = 0.1
    if (e < ocean_baseline): 
        return OCEAN
    if (e < ocean_baseline + 0.02): 
        return BEACH
    if (e > 0.8):
        if (m < 0.1): 
            return SCORCHED
        if (m < 0.2): 
            return BARE
        if (m < 0.5): 
            return TUNDRA
        return SNOW
    if (e > ocean_baseline + 0.5):
        if (m < 0.33): 
            return TEMPERATE_DESERT
        if (m < 0.66): 
            return SHRUBLAND
        return TAIGA
    if (e > ocean_baseline + 0.2):
        if (m < 0.16): 
            return TEMPERATE_DESERT
        if (m < 0.50): 
            return GRASSLAND
        if (m < 0.83): 
            return TEMPERATE_DECIDUOUS_FOREST
        return TEMPERATE_RAIN_FOREST
    if (m < 0.16): 
        return SUBTROPICAL_DESERT
    if (m < 0.33): 
        return GRASSLAND
    if (m < 0.66): 
        return TROPICAL_SEASONAL_FOREST
    return TROPICAL_RAIN_FOREST

Now that the code for turning noise into an interesting map was taken care of, the next step was to actually write the functions to generate noise. In the interest of speed to enable us to more quickly implement, refine, and test our project, and since the main point of this project was to evaluate quantum-noise-based procedurally generated maps anyway, I decided to use the highly optimized **Python noise** library instead of our manually implemented Perlin noise functions to serve as the basis for our classical noise generation examples. 

The **gen_elevation** function generates Perlin noise for elevation levels (the $e$ parameter of **biome_basic** and **biome_advanced**), while the **gen_moisture** function generates moisture noise (the $m$ parameter of **biome_basic** and **biome_advanced**).

In [8]:
def gen_perlin_noise(scale, freq, show_image = False):
    # Inputs: 
    #       scale: Integer - Larger -> Lower frequency -> More bumps and hills
    #       show_image: Boolean - True then show matplotlib image
    arr = np.zeros(scale)
    for x in range(scale[0]):
        for y in range(scale[1]):
            arr[x][y] = (python_noise.pnoise2(x/freq, 
                                        y/freq, 
                                        octaves=1, 
                                        persistence=0, # Irrelevant
                                        lacunarity=1, # Irrelevant
                                        repeatx=scale[0], 
                                        repeaty=scale[1], 
                                        base=0) + 1)/2
    if show_image:
        display_image(arr)
    return arr

def get_layered_noise(scale, levels, weights):
    # Inputs:
    #       levels: Array of Integers -> scale for each octavte
    #       weights: Array of Integers -> How much each octave is weighted
    s = 0
    for idx, l in enumerate(levels):
        s += weights[idx] * gen_perlin_noise(scale, l)
    return s

def gen_elevation(scale, valley_power = 3):
    # Inputs:
    #             valley_power: Integer - Larger -> More valleys (default 3)
    # Outputs:
    #            1024x1024 array
    # Elevation
    elevation = get_layered_noise(scale, [50,150,300], [1, .5, .1])
    # Redistribution for valleys
    elevation = elevation ** valley_power
    return elevation

def gen_moisture(scale):
    # Inputs:
    #             None
    # Outputs:
    #            1024x1024 array
    # Moisture
    moisture = get_layered_noise(scale, [50,200,400], [1, .5, .1])
    return moisture

An example of the **gen_elevation** function is defined below, with different parameter values for **valley_power** tested to show how changes in elevation and the creation of valleys (the dark pixels) can be controlled by exponentiation:

In [9]:
display_image(gen_elevation((1024,1024), valley_power = 3))
display_image(gen_elevation((1024,1024), valley_power = 2))
display_image(gen_elevation((1024,1024), valley_power = 1))

#### Elevation Noise

##### Valley = 3
![Elevation, Valley 3](Images/elevation_3.png)

##### Valley = 2
![Elevation, Valley 2](Images/elevation_2.png)

##### Valley = 1
![Elevation, Valley 1](Images/elevation_1.png)

Finally, having written all of the helper functions necessary in the above code segments, I implemented the code to actually generate images of maps. The function **make_map** takes as input either classically-generated or quantum-generated noise, then generates a map of either an island surrounded by water or a fully land-based terrain.

In [10]:
def gen_circle_grad(scale):
    # Inputs:
    #             None
    # Outputs:
    #            1024x1024 array of a circle gradient
    # Circle gradient - used for island generation
    center_x = scale[0] //2
    center_y = scale[1] // 2
    circle_grad = [ [ np.sqrt((x - center_x)**2 + (y - center_y)**2) for y in range(scale[1]) ] for x in range(scale[0]) ]
    ## Reshape and standardize to [-1,1]
    circle_grad /= np.max(circle_grad)
    circle_grad = -(circle_grad - 0.5)*2.0
    ## Overexaggerate center
    circle_grad = [ [ circle_grad[x][y] * (20 if circle_grad[x][y] > 0 else 1) for y in range(scale[1]) ] for x in range(scale[0]) ]
    circle_grad /= np.max(circle_grad)
    return circle_grad

def make_map(scale, elevation = None, moisture = None, island = True, advanced = True, display = True):
    # Inputs:
    #            scale: (Integer, Integer) - width (x) and height (y) of map
    #            elevation: 1024x1024 matrix - use instead of generating noise classically
    #            moisture: 1024x1024 matrix - use instead of generating noise classically
    #             island: Boolean - If true, then use circle gradient to make map an island
    #             advanced: Boolean - If true, then use moisture noise to color pixels
    #             display: Boolean - If true, then print out map
    # Outputs:
    #            1024x1024 array of colored world pixels
    elevation = gen_elevation(scale) if elevation is None else elevation
    biome_func = biome_basic
    if island:
        # Get circular gradient
        circle_grad = gen_circle_grad(scale)
        # Convert world noise to circle gradient
        elevation = [ [ elevation[x][y] * circle_grad[x][y] * (20 if (elevation[x][y] * circle_grad[x][y]) > 0 else 1) for y in range(scale[1]) ] for x in range(scale[0]) ]
    elevation /= np.max(elevation)
    if advanced:
        biome_func = biome_advanced
        moisture = gen_moisture(scale) if moisture is None else moisture
    if moisture is None:
        moisture = np.ones(scale)
    # Color world pixels
    world = [ [ biome_func(elevation[x][y], moisture[x][y]) for y in range(scale[1]) ] for x in range(scale[0]) ]
    if display:
        display_image(world)
    return world

In order to create an island and ensure that it is surrounded on all sides by water, the **make_map** function uses the **gen_circle_grad** function defined above. This function generates a circular gradient radiating from the center of the map. With higher values closer towards the center, this circular gradient ensures that pixels farther away from the center are down-shifted in value. And since my **biome_basic** and **biome_advanced** functions treat the lowest pixel values as **OCEAN**, the points farther away from the center of the map will get classified as below sea level. 

In [11]:
# Circle gradient
display_image(gen_circle_grad(CLASSICAL_SCALE))

# Island gradient = Circle gradient * Elevation noise
e = gen_elevation(CLASSICAL_SCALE)
c = gen_circle_grad(CLASSICAL_SCALE)
island_grad = [ [ e[x][y] * c[x][y] * (20 if (e[x][y] * c[x][y]) > 0 else 1) for y in range(CLASSICAL_SCALE[1]) ] for x in range(CLASSICAL_SCALE[0]) ]
island_grad /= np.max(island_grad)
display_image(island_grad)

#### Circle Gradient Noise

![Circle Gradient](Images/circle_grad.png)

#### Island Elevation = Circle Gradient * Elevation

![Island Gradient](Images/island_grad.png)

The code below generates four distinct maps using classical Perlin noise. 

Images of the four maps will pop-up in the background when this code chunk is run, and will appear in the following order: 

1. _Island_ - Biomes determined by elevation and moisture
1. _Island_ - Biomes determined by elevation only
1. _Land mass_ - Biomes determined by elevation and moisture
1. _Land mass_ - Biomes determined by elevation only

In [12]:
#
# Classical examples
#
a = make_map((1024, 1024), island = True, advanced = True)
a = make_map((1024, 1024), island = True, advanced = False)
a = make_map((1024, 1024), island = False, advanced = True)
a = make_map((1024, 1024), island = False, advanced = False)

#### Island

##### Advanced 
![Island Advanced](Images/classic_island_advanced.png)

##### Basic 
![Island Basic](Images/classic_island_basic.png)

#### Land

##### Advanced 
![Land Advanced](Images/classic_land_advanced_1.png)

##### Basic 
![Land Basic](Images/classic_land_basic_1.png)

## 4. Quantum Circuits (Michael/Vishal)

Building on the theory developed above concerning the procedural generation of interesting island/non-island maps with classical noise algorithms, I next decided to map the amplitudes of the states output from several quantum circuits to the map generating code above and use that output as noise for my maps.

This section relies heavily on the helper functions defined above, as well as IBM's QiSkit library for quantum circuit generation.

### A) Set-Up (Michael)

Given that the world's largest quantum computer currently only supports 72 qubits [https://thenextweb.com/artificial-intelligence/2018/03/06/google-reclaims-quantum-computer-crown-with-72-qubit-processor/] and the high computational cost for simulating quantum systems on classical computers, I decided to use **10 qubits** in the quantum circuits below for my map generation simulations.

In [13]:
# Number of qubits
N_QUBITS = 10
# Number of replications
shots = 4**N_QUBITS

# Default ground state
GROUND_STATE = [0] * (2**N_QUBITS)
GROUND_STATE[0] = 1

# Create max X and Y needed to squash 2^n points onto 2D plane (not necessarily square so ends up creating a perfect tiling)
max_X = int(2 ** np.ceil(N_QUBITS/2))
max_Y = int(2 ** np.floor(N_QUBITS/2))
SCALE = (max_X, max_Y)

In [14]:
# For each (x,y) on 2D plane...
coordinates = {} # Map (x,y) coordinate to binary string '1000101'
for x in range(SCALE[0]):
    for y in range(SCALE[1]):
        # For each qubit...
        for i in range(N_QUBITS):
            # Create binary string representation
            if (i%2) == 0:
                ## If even qubit
                xx = np.floor(x/2**(i/2))
                coordinates[(x,y)] = str( int( ( xx + np.floor(xx/2) )%2 ) ) + (coordinates[(x,y)] if (x,y) in coordinates else '')
            else:
                ## If odd qubit
                yy = np.floor(y/2**((i-1)/2))
                coordinates[(x,y)] = str( int( ( yy + np.floor(yy/2) )%2 ) ) + (coordinates[(x,y)] if (x,y) in coordinates else '')
#
# Now, "coordinates" is dictionary of form:
## { (2, 1): '000000111', (3, 1): '000000110' }
#

# Set ground state (000...000) to be in center of hypercube 
center = '0' * N_QUBITS
current_center = coordinates[ ( np.floor(SCALE[0]/2), np.floor(SCALE[1]/2)) ]
diff = ''
for j in range(N_QUBITS):
    diff += '0'*(current_center[j]==center[j]) + '1'*(current_center[j]!=center[j])
#
# Marks with "1" digits where current_center and center are different, "0" otherwise
## If center == '000' and current_center == '101', then diff = '101'
## If center == '110' and current_center == '101' then diff = '011'
# 

#
# Flip all bits based on "diff"
#
for (x,y) in coordinates:
    newstring = ''
    for j in range(N_QUBITS):
        newstring += coordinates[(x,y)][j]*(diff[j]=='0') + ('0'*(coordinates[(x,y)][j]=='1')+'1'*(coordinates[(x,y)][j]=='0'))*(diff[j]=='1')
    coordinates[(x,y)] = newstring

# Create inverse of 'coordinates' dictionary
reverse_coordinates = {} # Map binary string '1010101010' to (x,y) coordinate
for x in range(SCALE[0]):
    for y in range(SCALE[1]):
        reverse_coordinates[coordinates[(x,y)]] = (x,y)


### B) Helper Functions (Michael)

The function **get_probs_matrix** below takes as input a Qiskit job and retrieves the count of each of the $2^{N}$ possible states. It then divides each count by the number of total replications ("shots") that were done to get the MLE estimate for the true probability of each state.

It then uses the **reverse_coordinates** matrix defined above to map each state (returned by Qiskit as a binary string) to an $(x,y)$ coordinate on my map. It then associates that $(x,y)$ coordinate with the probability measured for that state, and thus the "noise value" for that pixel $(x,y)$ is simply the probability of the quantum state mapped to that pixel.

In [15]:
#
# Convert probabilities of each state after running a QC to a noise matrix
#
def get_probs_matrix(job):
    # Convert a QC job into a 1024x1024 noise matrix
    counts = job.result().get_counts()
    probs_matrix = np.full(SCALE, 1/shots)
    for binary_string, count in counts.items():
        probs_matrix[reverse_coordinates[binary_string]] = count/shots
    return probs_matrix

The helper functions below offer several useful pieces of functionality for setting up and evaluating quantum circuits. 

The **get_peaks** function returns a list of the $(x,y)$ coordinates with the highest values in a noise array (i.e. the states output from a quantum circuit with the highest associated probabilities). Visually, these $(x,y)$ coordinates correspond to the whitest pixels in the **display_image** rendering of that noise matrix.

The **create_superposition** function accepts one parameter, $density$, and returns a vector representing a superposition at that desired density. The lower the density, the fewer states will be included in that superposition. This function is useful for initializing our quantum circuits, for spreading out the probabilitiy density among many states through superposition allows us to better simulate the largely spread out distribution generated by the classical Perlin noise algorithm.

In [27]:
#
# Quantum circuit set-up code
#

#
# Get peaks of noise
def get_peaks(noise, side_peaks_threshold = None, side_peaks_num = None, display = True):
    # Inputs:
    #       noise: 1024x1024 matrix - From get_probs_matrix()
    #       side_peaks_threshold: Float - Any values in "noise" greater than this threshold will be counted as a "side_peak"
    #       side_peaks_num: Integer - Number of side_peaks to count (will return top "side_peaks_num" values in "noise")
    #       display: Boolean - If true, print out peak and side_peaks
    peak = np.unravel_index(a.argmax(), a.shape)
    if side_peaks_threshold is not None:
        side_peaks = np.argwhere( a > side_peaks_threshold)
    else:
        side_peaks = np.unravel_index(np.argsort(a.ravel())[-side_peaks_num:], a.shape)
        side_peaks = np.array(list(zip(side_peaks[0], side_peaks[1])))
    result = { 'peak' : coordinates[peak], 'side_peaks' : [ coordinates[(sp[0], sp[1])] for sp in list(side_peaks)] }
    if display:
        print(result['peak'])
        print(result['side_peaks'])
    return result

#
# Create superposition at desired density
def create_superposition(density):
    # Input:    density: Float between (0,1) - Larger -> More states in superposition (more spread probability)
    # Output:   state: Array of length 512 with Z elements that are of value 1/sqrt(Z)
    ##          "state" represents a uniform superposition in Z diff states
    total_states = 2**N_QUBITS # Assuming N_QUBITS = 9...
    n_states = int(density * total_states) # N = 51, number of states in superposition
    state = [0]* total_states # Array of 0's, len(state) = 512
    # Randomly pick N = 51 points (set by "n_states")
    super_states = np.random.choice(range(total_states), size = n_states, replace = False) # States in superposition
    for s in super_states:
        state[s] = 1
    normalization = np.sqrt(sum(np.absolute(state)**2)) # Sum of squared amplitudes
    state = [ amp / normalization for amp in state ]
    return state

The two functions below help make generation of a Qiskit quantum circuit easier. 

The **get_quantum_circuit** function simply returns registers, a circuit, and a QASM simulator backend.

The **sim_circuit** function serves as a decorator that automatically adds support for logarithmic smoothing, normalization, non-ground-state initialization, and visualization of the output of the quantum circuit it wraps.

In [17]:
#
# Helper function
def get_quantum_circuit():
    q = QuantumRegister(N_QUBITS)
    c = ClassicalRegister(N_QUBITS)
    qc = QuantumCircuit(q, c)
    backend = BasicAer.get_backend('qasm_simulator')
    return q, c, qc, backend

#
# Decorate for all circuits that don't rely on real-world device noise
def sim_circuit(func):
    def wrapper(*args, **kwargs):
        #
        # (1) Get backend
        q, c, qc, backend = get_quantum_circuit()
        #
        # (2) Make circuit
        initial_state = kwargs['initial_state'] if 'initial_state' in kwargs else GROUND_STATE
        func(q, c, qc, initial_state, **kwargs)
        #
        # (3) Execute
        job = execute(qc, backend, shots = shots)
        #
        # (4) Display
        log_smooth = kwargs['log_smooth'] if 'log_smooth' in kwargs else False
        normalize = kwargs['normalize'] if 'normalize' in kwargs else False
        display = kwargs['display'] if 'display' in kwargs else True # Default to True
        probs_matrix = get_probs_matrix(job)
        if log_smooth:
            ## Take log
            probs_matrix = -np.log(probs_matrix)
        if normalize:
            ## Normalize to be between [0,1]
            probs_matrix = (probs_matrix - np.min(probs_matrix))/(np.max(probs_matrix) - np.min(probs_matrix))
        if display:
            ## Display grainy image of noise
            display_image(probs_matrix)
        return probs_matrix
    return wrapper
def parse_args(args):
    # Parse args from @sim_circuit decorator
    return args[0], args[1], args[2], args[3]

### C) Circuits (Michael)

The code below actually implements quantum circuits for a variety of set-ups. The comments for each function described in more detail what circuit is being tested.

In [18]:
#
# Construct actual Quantum circuits
#

#
# Simple circuit (just measure ground state)
@sim_circuit
def simple_circuit(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    # Initialize circuit
    qc.initialize(initial_state, q)
    # Measure 
    qc.measure(q, c)
@sim_circuit

#
# GHZ Gate-based circuits
def simple_ghz_circuit(*args, **kwargs):
    # Output: Use GHZ state (generalization of Bell state) to show 2 distinct peaks
    q, c, qc, initial_state = parse_args(args)
    # Initialize circuit
    qc.initialize(initial_state, q)
    # GHZ gate
    qc.h(q[0])
    for i in range(N_QUBITS - 1):
        qc.cx(q[i], q[i+1])
    qc.measure(q, c)
@sim_circuit
def rotation_ghz_circuit(*args, **kwargs):
    # Spread out probability from 2 GHZ peaks to surrounding pixels
    # Inputs: rotation: Float - Closer to pi/2, more literal checkerboard
    ##      pi/2 => checkerboard, top left is white
    ##      -pi/2 => checkerboard, top left is black
    # Output: Variation on checkerboard pattern
    q, c, qc, initial_state = parse_args(args)
    rotation = kwargs['rotation']
    # Initialize circuit
    qc.initialize(initial_state, q)
    # GHZ gate
    qc.h(q[0])
    for i in range(N_QUBITS - 1):
        qc.cx(q[i], q[i+1])
    # RY rotation
    qc.ry(rotation, q)
    # Measure
    qc.measure(q, c)

#
# Lab 4 Gates
@sim_circuit
def lab_4_gate_1(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    for i in range(N_QUBITS):
        qc.h(q[i])
        qc.h(q[i])
    qc.measure(q, c)
@sim_circuit
def lab_4_gate_2(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    for i in range(N_QUBITS):
        qc.h(q[i])
        qc.t(q[i])
        qc.h(q[i])
    qc.measure(q, c)
@sim_circuit
def lab_4_gate_3(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    for i in range(N_QUBITS):
        qc.h(q[i])
        qc.s(q[i])
        qc.h(q[i])
    qc.measure(q, c)
@sim_circuit
def lab_4_gate_4(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    for i in range(N_QUBITS):
        qc.h(q[i])
        qc.s(q[i])
        qc.t(q[i])
        qc.h(q[i])
    qc.measure(q, c)
@sim_circuit
def lab_4_gate_5(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    for i in range(N_QUBITS):
        qc.h(q[i])
        qc.z(q[i])
        qc.h(q[i])
    qc.measure(q, c)

#
# Template circuit
@sim_circuit
def template_circuit(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    ##
    ##
    ## EDIT HERE
    ##
    ##
    qc.measure(q, c)

### D) Even More Circuits! (Vishal)

In [19]:
@sim_circuit
def qft_circuit(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    qc.h(q[0])
    
    # Phase gates
    qc.cu1(np.pi/2, q[0], q[1])
    qc.cu1(np.pi/4, q[0], q[2])
    
    qc.h(q[1])
    qc.cu1(np.pi/2, q[1], q[2])
    qc.h(q[2])
    qc.swap(q[0], q[2])
    
    qc.measure(q, c)

@sim_circuit
def fredkin_circuit(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    # Apply fredkin gate multiple times. 
    for i in range(N_QUBITS - 2):
        qc.cswap(q[i], q[i+1], q[i+2])
    qc.measure(q, c)
    
@sim_circuit
def toffoli_circuit(*args, **kwargs):
    q, c, qc, initial_state = parse_args(args)
    qc.initialize(initial_state, q)
    # Apply toffoli gate multiple times. 
    for i in range(N_QUBITS - 2):
        qc.ccx(q[i], q[i+1], q[i+2])
    qc.measure(q, c)

### E) Make Maps (Michael)

Having now defined a variety of quantum circuits, we wanted to visualize the type of noise that each generated and how it could translate to actual maps.

In [20]:
#
# Make example maps
#
a = rotation_ghz_circuit(rotation = np.pi*.25, log_smooth = True)
b = make_map(SCALE, elevation = a)
a = rotation_ghz_circuit(rotation = np.pi*.25, log_smooth = True, normalize = True)
b = make_map(SCALE, elevation = a)
a = rotation_ghz_circuit(rotation = np.pi*.25, log_smooth = False, normalize = False)
b = make_map(SCALE, elevation = a)

#### Island - Logarithmic smoothing
##### Map
![title](Images/example1_logsmooth_island.png)
##### Noise
![title](Images/example1_logsmooth_island_grad.png)
#### Island - Logarithmic smoothing + Normalization
##### Map
![title](Images/example1_logsmooth_norm_island.png)
##### Noise
![title](Images/example1_logsmooth_norm_island_grad.png)
#### Island - No smoothing or normalization
##### Map
![title](Images/example1_island_1000.png)
##### Noise
![title](Images/example1_island_grad_1000.png)

In [21]:
#
# Experiment with different rotations
#
for rot in np.linspace(0, np.pi/2, num = 5):
    a = rotation_ghz_circuit(rotation = rot, log_smooth = True, normalize = True)
    b = make_map(SCALE, elevation = a)

0.0
0.39269908169872414
0.7853981633974483
1.1780972450961724
1.5707963267948966


#### Rotation = $0$
##### Island
![title](Images/pi_1.png)
##### Noise
![title](Images/pi_1_grad.png)
#### Rotation = $0.39 \pi$
##### Island
![title](Images/pi_2.png)
##### Noise
![title](Images/pi_2_grad.png)
#### Rotation = $0.78 \pi$
##### Island
![title](Images/pi_3.png)
##### Noise
![title](Images/pi_3_grad.png)
#### Rotation = $1.17 \pi$
##### Island
![title](Images/pi_4.png)
##### Noise
![title](Images/pi_4_grad.png)
#### Rotation = $1.57 \pi$
##### Island
![title](Images/pi_5.png)
##### Noise
![title](Images/pi_5_grad.png)

In [22]:
#
# Experiment with different densities
#
# Island with craters and mini-lakes
a = rotation_ghz_circuit(rotation = np.pi/3, initial_state = create_superposition(0.1), log_smooth = True, normalize = False)
b = make_map(SCALE, elevation = a)
# Nice, circular island (b/c higher density, so smoother noise)
a = rotation_ghz_circuit(rotation = np.pi/3, initial_state = create_superposition(0.8), log_smooth = True, normalize = False)
b = make_map(SCALE, elevation = a)

#### Superposition density = 0.1, Log smoothing + Normalization
##### Island
![title](Images/craters_norm_island.png)
##### Noise
![title](Images/craters_norm_grad.png)
### Superposition density = 0.1, Log smoothing
##### Island
![title](Images/craters_island.png)
##### Noise
![title](Images/craters_grad.png)
### Superposition density = 0.8, Log smoothing + Normalization
##### Island
![title](Images/circular_island.png)
##### Noise
![title](Images/circular_grad.png)

As a sanity check, I also made a simple quantum circuit (no gates, just one measurement) to see whether the noise that was output was a single white dot in the center of the plot (since the center dot corresponds to the ground state '0000000' per my set-up code). If we do not see just a single white dot in the center of the plot, then something has gone terribly wrong.

In [23]:
#
# Check simple circuit is a single dot in the center
#
a = simple_circuit()
b = make_map(SCALE, elevation = a)

Thankfully, we do observe exactly what was expected with the **simple_circuit**. The map generated is a single white snowcap (since it has probability 1) amidst a literal ocean of blue pixels (since they have probability $0 < 0.1$).

#### Simple Circuit
##### Island
![title](Images/simple_circuit_island.png)
##### Noise
![title](Images/simple_circuit_grad.png)

I performed the same sanity check on the simple GHZ gate, expecting to see exactly two white dots amidst a sea of black pixels. Sure enough, this is exactly what we see:

In [24]:
# Check GHZ gate
a = simple_ghz_circuit()
b = make_map(SCALE, elevation = a)

#### Simple GHZ Circuit
##### Island
![title](Images/simple_ghz_circuit_island.png)
##### Noise
![title](Images/simple_ghz_circuit_grad.png)

Some more interesting island shapes and quantum circuits are included below:

In [25]:
#
# Cross island
#
a = rotation_ghz_circuit(rotation = np.pi*.3, log_smooth = True)
b = make_map(SCALE, elevation = a)
#
# Nice, circular island
#
a = lab_4_gate_3()
b = make_map(SCALE, elevation = a)

#### Cross island (GHZ gate)
##### Island
![title](Images/cross_island.png)
##### Noise
![title](Images/cross_grad.png)
#### Circular island (Lab 4, Gate 3)
##### Island
![title](Images/nice_circular_island.png)
##### Noise
![title](Images/nice_circular_grad.png)

Next, out of curiousity, I wanted to test what types of noise/maps that the gates included in Lab 4 would generate. I also included the histograms for their outputs below, in order to check whether our outputted grayscale noise plot matched what we'd expect based on their predicted outputs.

Thankfully, all of the generated maps make sense when associated with their relevant histograms. All 5 gates are detailed below.

In [28]:
#
# Lab 4 Gates
#
a = lab_4_gate_1()
b = make_map(SCALE, elevation = a)
print(get_peaks(a, side_peaks_threshold = 0.01))
a = lab_4_gate_2()
b = make_map(SCALE, elevation = a)
print(get_peaks(a, side_peaks_threshold = 0.01))
a = lab_4_gate_3()
b = make_map(SCALE, elevation = a)
print(get_peaks(a, side_peaks_num = 10))
a = lab_4_gate_4()
b = make_map(SCALE, elevation = a)
print(get_peaks(a, side_peaks_threshold = 0.01))
a = lab_4_gate_5()
b = make_map(SCALE, elevation = a)
print(get_peaks(a, side_peaks_threshold = 0.01))

0000000000
['0000000000']
{'peak': '0000000000', 'side_peaks': ['0000000000']}
0000000000
['0100000000', '1000000000', '0000000000', '0000000010', '0000001000', '0000100000', '0010000000', '0000000001', '0000000100', '0000010000', '0001000000']
{'peak': '0000000000', 'side_peaks': ['0100000000', '1000000000', '0000000000', '0000000010', '0000001000', '0000100000', '0010000000', '0000000001', '0000000100', '0000010000', '0001000000']}
0111100101
['0100000100', '0100010101', '0111011000', '1010101010', '0000001000', '1100101101', '0100001001', '0010010101', '0010101110', '0111100101']
{'peak': '0111100101', 'side_peaks': ['0100000100', '0100010101', '0111011000', '1010101010', '0000001000', '1100101101', '0100001001', '0010010101', '0010101110', '0111100101']}
1111111111
['1111101111', '1111111110', '1111011111', '1111111101', '1111111111', '1111110111', '1101111111', '0111111111', '1111111011', '1110111111', '1011111111']
{'peak': '1111111111', 'side_peaks': ['1111101111', '1111111110',

#### Gate 1
Gate 1 results in an output qubit of 0 with 100\% probability. Thus, we expect to see a single white pixel at the coordinate corresponding to '0000000000' (which, as previously mentioned, we've purposely set to be the central pixel in our noise plot).

##### Output
![title](Images/lab_4_gate_1.png)
##### Island
![title](Images/lab_4_gate_1_island.png)
##### Noise
![title](Images/lab_4_gate_1_grad.png)


#### Gate 2
Gate 1 results in an output state of 0 with much higher probability than a state of 1. We thus expect the state '0000000000' to be very white (the central pixel), but for there also to be slightly white pixels around it.

##### Output
![title](Images/lab_4_gate_2.png)
##### Island
![title](Images/lab_4_gate_2_island.png)
##### Noise
![title](Images/lab_4_gate_2_grad.png)


#### Gate 3
Gate 1 results in states 1 or 0 with about even probabilities. Thus, we expect our noise map to be have fairly gray/have a large spread of values.

##### Output
![title](Images/lab_4_gate_3.png)
##### Island
![title](Images/lab_4_gate_3_island.png)
##### Noise
![title](Images/lab_4_gate_3_grad.png)

#### Gate 4
Gate 1 results in an output state of 1 with much higher probability than a state of 0.

##### Output
![title](Images/lab_4_gate_4.png)
##### Island
![title](Images/lab_4_gate_4_island.png)
##### Noise
![title](Images/lab_4_gate_4_grad.png)

#### Gate 5
Gate 5 results in an output state of 1 with 100\% probability. Thus, we expect to see a single white pixel at the coordinate corresponding to '1111111111'. This white pixel should be (and is) the same as the most white pixel in the Gate 4 plot.

##### Output
![title](Images/lab_4_gate_5.png)
##### Island
![title](Images/lab_4_gate_5_island_1.png)
##### Noise
![title](Images/lab_4_gate_5_grad.png)

### F) Even More Maps! (Vishal)

**NOTE:** Images for these are not included because we generated them live as part of our presentation demo.

In [29]:
a = fredkin_circuit(initial_state = create_superposition(0.5), log_smooth = True)
b = make_map(SCALE, elevation = a)

a = toffoli_circuit(initial_state = create_superposition(0.5), log_smooth = True)
b = make_map(SCALE, elevation = a)

a = qft_circuit(initial_state = create_superposition(0.5), log_smooth = True)
b = make_map(SCALE, elevation = a)

## G) Quantum Noise (Michael)

The last thing we wanted to test, in addition to the interesting patterns that quantum gates in and of themselves could generate (as shown above), was generating noise from the actual measurement error of dealing with real-world quantum devices. This measurement error can be thought of as an "extra layer" of unpredictable yet natural noise that we get as a bonus for using quantum circuits to generate our maps.

The code below runs the Lab 4 Gate 3 quantum circuit in conjunction with the measurement noise associated with running that circuit on a real-world device. It then creates a non-island map. 

In [ ]:
#########
## NOTE: This only works if running in an environment that supports qiskit.providers.aer.noise
## Otherwise, this will throw a "No module named 'qasm_controller_wrapper'" error
#########

from qiskit.providers.aer import noise

IBMQ.save_account('1527ec48a8f286efe0275e278f2de4c2c66510b08e13b54d4b1a89626d0a38c3419107fa0d44998bb5473b12583e474d568dcfb1e8763fdb689160df3781b26f', overwrite = True)
IBMQ.load_accounts()
backend_to_simulate = IBMQ.get_backend('ibmq_16_melbourne')
noise_model = noise.device.basic_device_noise_model(backend_to_simulate.properties())

q, c, qc, backend = get_quantum_circuit()
# Lab 4, Gate 3
for i in range(N_QUBITS):
    qc.h(q[i])
    qc.s(q[i])
    qc.h(q[i])
qc.measure(q, c)

job = execute(qc, backend, shots=shots, noise_model=noise_model, basis_gates=noise_model.basis_gates)
elevation = get_probs_matrix(job)
elevation /= np.max(elevation)
elevation = -np.log(elevation)
display_image(elevation)
make_map(SCALE, elevation = elevation, moisture = elevation, island = False, advanced = False)

#### Land Map
![title](Images/quant_noise_land.png)
#### Island Map
![title](Images/quant_noise_island.png)
#### Noise
![title](Images/quant_noise_grad.png)

# 5. Evaluation (Michael)

Finally, I wanted to be able to actually quantify the differences between the aforementioned quantum circuits using some sort of standard, robust metric.

The evaluation criteria that I decided to utilize was **estimate_sigma** from the **Scikit-Image** library, which uses wavelet shrinkage to estimate the standard deviation of the noise in an image (in this case, simply a matrix of grayscale pixels). [D. L. Donoho and I. M. Johnstone. “Ideal spatial adaptation by wavelet shrinkage.” Biometrika 81.3 (1994): 425-455. DOI:10.1093/biomet/81.3.425]. The higher this value, the more "noise" in the noise I generated, which could be either good or bad depending on the type of terrain a game designer wanted to create -- a higher value would mean rougher terrain, while a lower value meant smoother terrain.

In [31]:
#
# Measure noise of noise matrix
def calc_noise(noise):
    # Citation: D. L. Donoho and I. M. Johnstone. “Ideal spatial adaptation by wavelet shrinkage.” Biometrika 81.3 (1994): 425-455. DOI:10.1093/biomet/81.3.425
    # Inputs:   noise: 1024x1024 matrix
    # Outputs:  sigma: Float - Larger -> More variance in noise
    sigma = estimate_sigma(noise, multichannel = False)
    return sigma

I wanted to see how the density passed to **create_superposition** changed the variation in the noise generated by a quantum circuit using this superposition as its intial state. 

Intuitively, I expected a more dense superposition (and thus probability spread around more states) to have lower overall variation than a lower density superposition, for a higher density would mean that more of the probability was spread around and thus there wouldn't be dramatic variation in pixel values.

In [ ]:
%matplotlib inline

#
# Make graph of how density affects variance in noise
#
sigmas = []
x_range = np.linspace(0.01, 1, num = 10)
for i in x_range:
    a_s = []
    for j in range(2):
        # Average over 2 trials
        a = rotation_ghz_circuit(rotation = np.pi/3, initial_state = create_superposition(i), log_smooth = False, normalize = False, display = False)
        a_s.append(calc_noise(a))
    sigmas.append(np.mean(a_s))

plt.plot(x_range, sigmas, label = 'Quantum GHZ Rotation')
plt.ylabel('Sigma')
plt.xlabel('Superposition Density')
plt.legend(loc='upper right')
plt.show()

As the below run shows, my intuition was correct -- there is experimental evidence for the fact that increasing the number of states in the superposition decreases the roughness of the terrain generated.

![title](Images/sigma_1_1.png)

The same trend holds for the Lab 4 Gate 3.

In [ ]:
#
# Make graph of how density affects variance in noise
#
sigmas = []
x_range = np.linspace(0.01, 1, num = 10)
for i in x_range:
    a_s = []
    for j in range(2):
        # Average over 2 trials
        a = lab_4_gate_3(initial_state = create_superposition(i), log_smooth = False, normalize = False, display = False)
        a_s.append(calc_noise(a))
    sigmas.append(np.mean(a_s))

plt.plot(x_range, sigmas, label = 'Lab 4 Gate 3')
plt.ylabel('Sigma')
plt.xlabel('Superposition Density')
plt.legend(loc='upper right')
plt.show()

![title](Images/lab_4_sigma_1.png)

# 6. Conclusion

We were able to successfully demonstrate that quantum circuits can be used to procedurally generate interesting, natural-looking maps and terrains for video games. Though we were only able to simulate 32 by 32 bit maps (since we used 10 qubits), the outputs that we did get perfectly matched our intuition and the expected theoretical properties of those circuits.

In terms of future work, it would have been nice to be able to directly implement Perlin noise in a quantum circuit, rather than using fairly arbitrarily selected quantum circuits as the basis for our noise function. Interestingly, however, even though the quantum circuits that we used are quite simple and contrived, the noise that they generate is able to (at least visually) match if not beat the noise generated by the classical Perlin noise algorithms in terms of generating natural-looking maps. Additionally, when the measurement noise of an actual real-world quantum computer running these quantum circuits is factored in, we essentially get an additional "layer" of truly natural noise for free, which could give game developers a novel method for creating even more interesting maps.

Notes on who contributed what portions of the project are included as parentheticals besides each section.

This project was inspired by several papers mentioned previously in this text, IBM's QiSkit tutorials, and video game development walkthroughs available online. Links to relevant sources for further information are included below.

	https://shanee.io/blog/2015/09/25/procedural-island-generation/
	http://flafla2.github.io/2014/08/09/perlinnoise.html
	https://www.redblobgames.com/articles/noise/introduction.html
	https://www.redblobgames.com/maps/terrain-from-noise/
    https://nbviewer.jupyter.org/github/Qiskit/qiskit-tutorial/blob/master/community/games/random_terrain_generation.ipynb
	http://jcgt.org/published/0004/02/01/paper.pdf
	https://medium.com/@yvanscher/playing-with-perlin-noise-generating-realistic-archipelagos-b59f004d8401
	http://bit-player.org/2011/a-slight-discrepancy
   
Thank you for a great semester!